In [1]:
HSMPY_PATH = r'C:\Users\MR068144\Downloads\Python Scripts'
# importing required packages
import sys, os, arcpy, csv, json, math
sys.path.append(HSMPY_PATH)
import hsmpy
import arcpy
import shutil
import os

Years = range(2017,2018)
IRISPath = r'\\CHCFPP01\Proj\ILDOT\650511SAFETYPROGRAM\7_BaseData\Master_BaseFiles_Data\5.0_Roadway_Layer\IRIS'
IRISPath = r'C:\Users\mr068144\Downloads\IRIS\IDOT Zip Files'
IRIS_Route = {y:os.path.join(IRISPath,'HWY'+str(y)+'_route.shp') for y in Years}
IRIS_Tab   = {y:os.path.join(IRISPath,'HWY'+str(y)+'_table.dbf') for y in Years}
WDir = r'C:\Users\mr068144\Downloads\Curvature'
#WDir = r'\\CHCFPP01\Proj\ILDOT\650511SAFETYPROGRAM\7_BaseData\Master_BaseFiles_Data\8.0_Curves_Layer'

In [ ]:
for year in range(2011,2017):
    print(year)
    FN = 'Curves_' + str(year) + '.mdb'
    MDB = os.path.join(WDir,FN)
    Overlay_Table = os.path.join(MDB , 'HWY' + str(year) + '_HCurve_diss')
    OverlayLayer = hsmpy.common.CreateOutPath(MainFile = MDB + '\\HWY' + str(year),appendix='HCurve1',Extension='')
    IRIS_Routes = IRIS_Route[year]
    F = [f.name for f in arcpy.ListFields(Overlay_Table) if not f.required and not f.name in ['INVENTORY','BEG_STA','END_STA','CMF_CH18']]
    hsmpy.network.CreateRouteEventLayer(Sites_Routes=IRIS_Routes,AttTable=Overlay_Table,
                                        BMP='BEG_STA',EMP='END_STA',RouteID='INVENTORY',Fields=F,
                                        Output=OverlayLayer)


In [2]:
subp = []
#Years = range(2005,2017)
for year in Years:
    FN = 'Curves_' + str(year) + '.mdb'
    MDB = os.path.join(WDir,FN)
    try: os.remove(MDB)
    except: pass
    try: arcpy.CreatePersonalGDB_management(out_folder_path=WDir,out_name=FN)
    except: pass   
    CurveTable = hsmpy.common.CreateOutPath(MainFile = MDB + '\\Curves_Table',appendix=str(year),Extension='')
    CurveLayer = hsmpy.common.CreateOutPath(MainFile = MDB + '\\Curves_Seg',appendix=str(year),Extension='')
    OutDir = os.path.dirname(IRIS_Route[year])
    OverlayTable = hsmpy.common.CreateOutPath(MainFile = MDB + '\\HWY' + str(year) + '_table',appendix='HCurve',Extension='')
    OverlayLayer = hsmpy.common.CreateOutPath(MainFile = MDB + '\\HWY' + str(year),appendix='HCurve',Extension='')
    subp.append(hsmpy.network.CON_ExtractCurves(WDir,HSMPY_PATH,
                                                IRIS_Route[year],IRIS_Tab[year],
                                                CurveTable,CurveLayer,
                                                OverlayTable,OverlayLayer,
                                                year))

In [10]:
import subprocess
subp = []
for y in [2011]:
    subp.append(subprocess.Popen([sys.executable, WDir + r'\CurveExtract_' + str(y)+ '.py'],shell=False,creationflags = subprocess.CREATE_NEW_CONSOLE))